In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

source_folder = Path("atp_transformed/2000-2024 with covid.csv")

In [3]:
df = pd.read_csv(source_folder)

C:\Users\Golbe\AppData\Local\Temp\ipykernel_40464\1506103381.py:1: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_folder)


In [4]:
# make everything date time
df["tourney_date"] = pd.to_datetime(df["tourney_date"])

# make a column with the years
df["tourney_year"] = df["tourney_date"].dt.year

# get the career length
df_years = (
    df.groupby("player_id")["tourney_year"]
      .agg(first_season="min", last_season="max")
)

# create the years active of the player
df_years["years_active"] = df_years["last_season"] - df_years["first_season"] + 1

In [5]:

# get the static data for all players
df_statics = df[["player_id","player_name","player_height", "player_country"]]
df_statics = df_statics.drop_duplicates("player_id")

In [6]:
# create the total values for all players
df_means = (
    df.groupby("player_id")[['minutes','player_rank_points', 'ace', 'double_faults',
                               'points_on_serve','first_serve_in', '1st_won', '2nd_won', 'player_age',
                               'service_games','break_points_saved', 'break_points_faced', 'match_outcome']]
            .mean().round()
            )

In [7]:
df_match_count = df.groupby("player_id").agg(matches_played=("match_outcome","count"))

In [8]:
df_max_rank = df.groupby("player_id").agg(worst_rank=("player_rank","max"),best_rank=("player_rank","min")).round(0)
df_max_rank["rank_differential"] = df_max_rank["worst_rank"] - df_max_rank["best_rank"]

In [9]:
df_merged = (df_statics
                .merge(df_means, on="player_id",how="left")
                .merge(df_years, on="player_id",how="left")
                .merge(df_match_count, on="player_id",how="left")
                .merge(df_max_rank, on="player_id",how="left")
             )

df_merged.to_csv(Path("atp_transformed/2000-2024_career_averages.csv"),index=False)